In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xsensdeviceapi as xda
import sys
from threading import Lock
import re
import time

In [2]:
def stopAll():
    global device
    global control
    global Ports
    print("Arrêt de l'enregistrement, mise du dispositif en mode configuration.")
    device.stopRecording()
    device.gotoConfig()
    device.disableRadio()
    print("Fermeture du fichier mtb")
    device.closeLogFile()
    print("Fermeture du port")
    control.closePort(Ports)
    control.close()
    xda.XsControl.destruct(control)
    
def checkConnectedSensors(devIdAll):
    global children
    global control
    global device
    childUsed = np.full(np.shape(children),False).tolist()
    if children.empty():
        print("Aucun dispositif trouvé.")
        stopAll()
        raise RuntimeError("Échec. Aucun capteur MTw trouvé.")
    else:
        for i in range(len(children)):
            if children[i].connectivityState() == xda.XCS_Wireless:
                childUsed[i] = True
        
        print("Dispositifs rejetés:")
        rejects = np.array(devIdAll)[[not elem for elem in childUsed]].tolist()
        for i in range(len(rejects)):
            index = devIdAll.index(rejects[i])
            print("%d - "%index +"%s" %rejects[i])
        
        print("Dispositifs acceptés:")
        accepted = np.array(devIdAll)[childUsed].tolist()
        for i in range(len(accepted)):
            index = devIdAll.index(accepted[i])
            print("%d - "%index +"%s" %accepted[i])
        
        option = str(input('Keep current status?'+' (y/n): ')).lower().strip()
        change = []
        if option[0] == 'n':
            op2 = input("\n Type the numbers of the sensors (csv list, e.g. 1,2,3) from which status should be changed \n (if accepted than reject or the other way around):\n")
            change = [int(i) for i in re.split(",",op2)]
            for i in range(len(change)):
                if a[change[i]]:
                    device.rejectConnection(children[change[i]])
                    childUsed[change[i]] = False
                else:
                    device.acceptConnection(children[change[i]])
                    childUsed[change[i]] = True
        if sum(childUsed) == 0:
            stopAll()
            raise RuntimeError("Échec. Aucun capteur MTw trouvé.")
    devicesUsed = np.array(children)[childUsed].tolist()
    devIdUsed = np.array(devIdAll)[childUsed].tolist()
    nDevs = sum(childUsed)

    return devicesUsed, devIdUsed, nDevs



class XdaCallback(xda.XsCallback):
    def __init__(self,max_buffer_size=5):
        xda.XsCallback.__init__(self)
        self.m_maxNumberOfPacketsInBuffer = max_buffer_size
        self.m_packetBuffer = list()
        self.m_lock = Lock()

    def packetAvailable(self):
        self.m_lock.acquire()
        res = len(self.m_packetBuffer) > 0
        self.m_lock.release()
        return res

    def onLiveDataAvailable(self, dev, packet):
        self.m_lock.acquire()
        assert(packet is not 0)
        while len(self.m_packetBuffer) >= self.m_maxNumberOfPacketsInBuffer:
            self.m_packetBuffer.pop()
        self.m_packetBuffer.append(xda.XsDataPacket(packet))
        self.m_lock.release()

In [ ]:
class MtwCallback(xda.XsCallback):
    def __init__(self,mtwIndex,device):
        self.m_mtwIndex = mtwIndex
        self.m_device =  device
        self.m_packetBuffer = list()
        self.m_lock = Lock()
    
    def packetAvailable(self):
        self.m_lock.acquire()
        res = len(self.m_packetBuffer) > 0
        self.m_lock.release()
        return res

    def getOldestPacket(self):
        self.m_lock.acquire()
        packet = self.m_packetBuffer[0]
        return packet

    def deleteOldestPacket(self):
        self.m_lock.acquire()
        self.m_packetBuffer.pop(0)
        self.m_lock.release()

    def onLiveDataAvailable(self, device, packet):
        self.m_lock.acquire()
        assert(packet is not 0)
        while len(self.m_packetBuffer) >= self.m_maxNumberOfPacketsInBuffer:
            self.m_packetBuffer.pop()
        self.m_packetBuffer.append(xda.XsDataPacket(packet))
        self.m_lock.release()   



In [3]:
try:
    # Extraire la version du Xs Device Api employé

    print("Creation d'un objet XsControl...")
    control = xda.XsControl_construct()
    assert(control is not 0)

    xdaVersion = xda.XsVersion()
    xda.xdaVersion(xdaVersion)
    print("Utilisation de la version XDA: %s \n" % xdaVersion.toXsString())


    print("Détection des ports#...\n")
    ports_Scan = xda.XsScanner_scanPorts(0,1
    ports_Scan = xda.XsScanner_scanPorts()00,True,True)

    # Détection des dispositifs MTw ou Station Awinda
    Ports = xda.XsPortInfo()
    for i in ange(poran[i].deviceId().i
        if ports_Scan[i].deviceId().isWirelessMaster() or ports_Scan[i].deviceId().isAwindaXStati
            on():
            P ports_Scan[i
     ]
            break

    if Ports.empty():
        raise RuntimeError("Échec. Aucun dispositif a été detecté.")

    devId = Ports.deviceId()
    print(" Dispositif trouvé avec les caractéristiques suivantes:")
    print(" Device ID: %s" % devId.toXsString())
    print(" Nom du port: %s" % Ports.portName())
    print(" Port baudrate: %s" % Ports.baudrate())

    print("Ouverture du port...")
    if not co ntrol.openPort(Ports.portName(),Ports.baudrate()):
        raise RuntimeError("Échec de l'ouverture du port.")

    device = control.device(devId)

    assert(device is not 0)
    print("Dispositif: %s, avec ID: %s ouvert. \n" %(device.pd().toX)St # Initialisation du Handlerring()))

    callback = XdaCallback()
    
    device.addCallbackHandler(callback)

    # Mettre le dispositif en mode configuration
    print("Dispositif en mode configuration...\n")
    if not device.gotoConfig():
        raise RuntimeError("Échec. Le dispositif n'a s pu êtset.")

   #device.se + xda.XSO_RetainLiveDatatO + xda.XSO_RetainLiveDataptions(x#daXSO_Orientation + xda.XSO_Calibrate, 0)

    c#onfigArray = xda.XsOutputConfgurationArray()
    configArray.push_back(xda.Xs#OutputConfiguration(xda.XDI_PacketCounter, 0))
    configArray.push_back(xda.XsOutputConfiguration(xda.XDI_SampleTimeFine, 0))
    #if device.deviceId().isAwindaXStation():
        #configArray.push_back(xda.XsOutputConfiguration(xda.XDI_Acceleration, 100))
        #configArray.push_back(xda.XsOutputConfiguration(xda.XDI_RateOfTurn, 100))
        #configArray.push_back(xda.XsOutputConfiguration(xda.XDI_MagneticField, 100))
        #configArray.push_back(xda.XsOutputConfiguration(xda.XDI_RotationMatrix, 100))
    #elif device.deviceId().isVru() or device.deviceId().isAhrs():
        #configArray.push_back(xda.XsOutputConfiguration(xda.XDI_Quaternion, 100))
    #elif device.deviceId().isGnss():
        #configArray.push_back(xda.XsOutputConfiguration(xda.XDI_Quaternion, 100))
        #configArray.push_back(xda.XsOutputConfiguration(xda.XDI_LatLon, 100))
        #configArray.push_back(xda.XsOutputConfiguration(xda.XDI_AltitudeEllipsoid, 100))
        #configArray.push_back(xda.XsOutputConfiguration(xda.XDI_VelocityXYZ, 100))
    #else:
        #rais#e RuntimeError("Unknown device while configuring. Aborting.#")
    if not device.setOutputConfiguration(configArray):
        raise RuntimeError("Could not configure the device. Aborting.")



    # Acquisition des fréquences pour la mise à jour des données
    fq_support = device.supportedUpdateRates(xda.XDI_None)
    index_fq = []

    while index_fq == []:
        print("Les fréquences disponibles sont:")
        print('\n'.join('{}: {}'.format(*k) for k in enumerate(fq_support)))
        choix_fq = input("Veuillez choisir une fréquence d'acquisition.")
        if choix_fq == []:
            continue
        index_fq = fq_support.index(int(choix_fq))
    print("Fréquence d'acquisition choisi: ", fq_support[index_fq], "\n")

    # Mise à jour de la fréquence d'acquisition
    if not device.setUpdateRate(fq_support[index_fq]):
        raise RuntimeError("Échec de la mise à jour du update rate.")

    
    # Acquisition du canal radio
    canaux_radio = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
    index_radio = []

    while index_radio == []:
        print("Les canaux disponibles sont: ")
        print('\n'.join('{}: {}'.format(*k) for k in enumerate(canaux_radio)))
        choix_radio = input("Veuillez choisir un canal de radio.")
        if choix_radio == []:
            continue
        index_radio = canaux_
radio.index(int(choix_radio))
    print("Canal de radio choisi: ", canaux_radio[index_radio], "\n")
    # Mise à jour du canal radio
    try:
        device.enableRadio(canaux_radio[index_radio])
    except ValueError:
        print("La radio est encore activée, veuillez débrancher l'appareil de l'ordinateur et reessayer.")


    input('\n Undock the MTw devices from the Awinda station and wait until the devices are connected (synced leds), then press enter... \n')

    # Vérifier les dispositifs détectées (nombre de capteurs MTw)
    children = device.children()

    devIdAll = []
    for i in range(len(children)):
        devIdAll.append(children[i].deviceId()) 

    devicesUsed,devIdUsed,nDevs = checkConnectedSensors(devIdAll)

    ##################

    print("Creating a log file...\n")
    logFileName = "logfile.mtb"
    if device.createLogFile(logFileName) != xda.XRV_OK:
        raise RuntimeError("Failed to create a log file. Aborting.")
    else:
        print("Created a log file: %de...\n")
    
      
    packet = device.takeFirstDataPacketInQueue()
    print("First data packet: ", packet.calibratedData().m_acc)

    
    packet = device.takeFirstDataPacketInQueue()
    packet_count = device.getDataPacketCount()
    print("First data packet: ", packet.calibratedData().m_acc, "\n")
    print("First data packet count:", packet_count, "\n")
e...\n")
    if not device.gotoMeasurement():
        raise RuntimeError("Could not put device into measurement mode. Aborting.")

    print("Starting recording...\n")
    if not device.startRecording():
        raise RuntimeError("Failed to start recording. Aborting.") # TODO ERREUR DANS LE LANCEMENT DE L'ENREGISTREMENT DES DONNÉES

    print("Main loop. Recording data for 10 seconds.") # Enregistrement des données durant 10 secondes

    startTime = xda.XsTimeStamp_nowMs()
    while xda.XsTimeStamp_nowMs() - startTime <= 10000:
        if callback.packetAvailable():
            # Retrieve a packet
            packet = callback.getNextPacket()
            
            s = ""
            
            if packet.containsCalibratedData():
                acc = packet.calibratedAcceleration()
                s = "Acc X: %.2f" % acc[0] + ", Acc Y: %.2f" % acc[1] + ", Acc Z: %.2f" % acc[2]
                
                gyr = packet.calibratedGyroscopeData()
                s += " |Gyr X: %.2f" % gyr[0] + ", Gyr Y: %.2f" % gyr[1] + ", Gyr Z: %.2f" % gyr[2]
                
                mag = packet.calibratedMagneticField()
                s += " |Mag X: %.2f" % mag[0] + ", Mag Y: %.2f" % mag[1] + ", Mag Z: %.2f" % mag[2]
                
            if packet.containsOrientation():
                quaternion = packet.orientationQuaternion()
                s = "q0: %.2f" % quaternion[0] + ", q1: %.2f" % quaternion[1] + ", q2: %.2f" % quaternion[2] + ", q3: %.2f " % quaternion[3]
                
                euler = packet.orientationEuler()
                s += " |Roll: %.2f" % euler.x() + ", Pitch: %.2f" % euler.y() + ", Yaw: %.2f " % euler.z()
                
            if packet.containsLatitudeLongitude():
                latlon = packet.latitudeLongitude()
                s += " |Lat: %7.2f" % latlon[0] + ", Lon: %7.2f " % latlon[1]
                    
            if packet.containsAltitude():
                s += " |Alt: %7.2f " % packet.altitude()
                    
            if packet.containsVelocity():
                vel = packet.velocity(xda.XDI_CoordSysEnu)
                s += " |E: %7.2f" % vel[0] + ", N: %7.2f" % vel[1] + ", U: %7.2f " % vel[2]

            print("%s\r" % s, end="", flush=True)


    print("\n Stopping recording...")
    if not device.stopRecording():
        raise RuntimeError("Failed to stop recording. Aborting.")

    print("Closing log file...")
    if not device.closeLogFile():
        raise RuntimeError("Failed to close log file. Aborting.")

    print("Removing callback handler...")
    device.removeCallbackHandler(callback)

    ##################

    exit = int(input('\n Appuyez sur la touche 0 pour quitter le mode acquisition... \n'))
    if exit == 0:
        stopAll()
    
except RuntimeError as error:
    print(error)
    sys.exit(1)

except:
    print("An unknown fatal error has occured. Aborting.")
    sys.exit(1)
    
else:
    print("Successful exit.")
    

Creation d'un objet XsControl...
Utilisation de la version XDA: 2021.0.0 build 5908 rev 109668 built on 2021-04-19 

Détection des ports...

 Dispositif trouvé avec les caractéristiques suivantes:
 Device ID: 01200816
 Nom du port: COM3
 Port baudrate: 2000000
Ouverture du port...
Dispositif: AW-A2, avec ID: 01200816 ouvert. 

Dispositif en mode configuration...

Les fréquences disponibles sont:
0: 120
1: 100
2: 80
3: 60
4: 40
Fréquence d'acquisition choisi:  120 

Les canaux disponibles sont: 
0: 11
1: 12
2: 13
3: 14
4: 15
5: 16
6: 17
7: 18
8: 19
9: 20
10: 21
11: 22
12: 23
13: 24
14: 25
Canal de radio choisi:  11 

Dispositifs rejetés:
Dispositifs acceptés:
0 - 00B43CDD
Creating a log file...

Created a log file: logfile.mtb

Putting devices into measurement mode...

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

First data packet:  [6.23037317e-307 8.45562019e-307 1.27949980e-307]
Starting recording...

Failed to start record

TypeError: object of type 'NoneType' has no len()

In [20]:
try:
    # Extraire la version du Xs Device Api employé

    print("Creation d'un objet XsControl...")
    control = xda.XsControl_construct()
    assert(control is not 0)

    xdaVersion = xda.XsVersion()
    xda.xdaVersion(xdaVersion)
    print("Utilisation de la version XDA: %s \n" % xdaVersion.toXsString())


    print("Détection des ports#...\n")
    ports_Scan = xda.XsScanner_scanPorts(0,100,True,True)

    # Détection des dispositifs MTw ou Station Awinda
    Ports = xda.XsPortInfo()
    for i in range(ports_Scan.size()):
        if ports_Scan[i].deviceId().isWirelessMaster() or ports_Scan[i].deviceId().isAwindaXStation():
            
            Ports = ports_Scan[i]
            break

    if Ports.empty():
        raise RuntimeError("Échec. Aucun dispositif a été detecté.")

    devId = Ports.deviceId()
    print(" Dispositif trouvé avec les caractéristiques suivantes:")
    print(" Device ID: %s" % devId.toXsString())
    print(" Nom du port: %s" % Ports.portName())
    print(" Port baudrate: %s" % Ports.baudrate())

    print("Ouverture du port...")
    if not control.openPort(Ports.portName(), Ports.baudrate()):
        raise RuntimeError("Échec de l'ouverture du port.")

    device = control.device(devId)

    assert(device is not 0)
    print("Dispositif: %s, avec ID: %s ouvert. \n" %(device.productCode(), device.deviceId().toXsString()))

    callback = XdaCallback()
    
    device.addCallbackHandler(callback)

    # Mettre le dispositif en mode configuration
    print("Dispositif en mode configuration...\n")
    if not device.gotoConfig():
        raise RuntimeError("Échec. Le dispositif n'a as pu être mis en mode configuration.")

    device.setOptions(xda.XSO_Orientation + xda.XSO_Calibrate + xda.XSO_RetainLiveData, 0)


    # Acquisition des fréquences pour la mise à jour des données
    fq_support = device.supportedUpdateRates(xda.XDI_None)
    index_fq = []

    while index_fq == []:
        print("Les fréquences disponibles sont:")
        print('\n'.join('{}: {}'.format(*k) for k in enumerate(fq_support)))
        choix_fq = input("Veuillez choisir une fréquence d'acquisition.")
        if choix_fq == []:
            continue
        index_fq = fq_support.index(int(choix_fq))
    print("Fréquence d'acquisition choisi: ", fq_support[index_fq], "\n")

    # Mise à jour de la fréquence d'acquisition
    if not device.setUpdateRate(fq_support[index_fq]):
        raise RuntimeError("Échec de la mise à jour du update rate.")

    
    # Acquisition du canal radio
    canaux_radio = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
    index_radio = []

    while index_radio == []:
        print("Les canaux disponibles sont: ")
        print('\n'.join('{}: {}'.format(*k) for k in enumerate(canaux_radio)))
        choix_radio = input("Veuillez choisir un canal de radio.")
        if choix_radio == []:
            continue
        index_radio = canaux_radio.index(int(choix_radio))
    print("Canal de radio choisi: ", canaux_radio[index_radio], "\n")

    # Mise à jour du canal radio
    try:
        device.enableRadio(canaux_radio[index_radio])
    except ValueError:
        print("La radio est encore activée, veuillez débrancher l'appareil de l'ordinateur et reessayer.")


    input('\n Undock the MTw devices from the Awinda station and wait until the devices are connected (synced leds), then press enter... \n')

    # Vérifier les dispositifs détectées (nombre de capteurs MTw)
    children = device.children()

    devIdAll = []
    for i in range(len(children)):
        devIdAll.append(children[i].deviceId()) 

    devicesUsed,devIdUsed,nDevs = checkConnectedSensors(devIdAll)

    ##################

    # Mis les capteurs MTw en mode acquisition des données
    print("Putting devices into measurement mode...\n")
    
    if not device.gotoMeasurement():
        raise RuntimeError("Could not put device into measurement mode. Aborting.")
    
    packet = children[0].takeFirstDataPacketInQueue()
    print("First data packet: ", packet.calibratedData().m_acc, "\n")
    print("Rotation matrix:", packet.orientationMatrix(), "\n")
    print("Waiting 10 seconds before acquiring data...\n")
    time.sleep(10)
    print("Starting acquisition...\n")
    
    time.sleep(5)
    packet2_count = children[0].getDataPacketCount()
    packet2 = children[0].getDataPacketByIndex(packet2_count)
    
    print("Other data packet: ", packet2.calibratedData().m_acc, "\n")
    print("Rotation matrix: ", packet2.orientationMatrix(), "\n")
    print("Packet count: ", packet2_count)



    print("Removing callback handler...")
    device.removeCallbackHandler(callback)


    ##################

    exit = int(input('\n Appuyez sur la touche 0 pour quitter le mode acquisition... \n'))
    if exit == 0:
        stopAll()
    
except RuntimeError as error:
    print(error)
    sys.exit(1)

except:
    print("An unknown fatal error has occured. Aborting.")
    sys.exit(1)
    
else:
    print("Successful exit.")

Creation d'un objet XsControl...
Utilisation de la version XDA: 2021.0.0 build 5908 rev 109668 built on 2021-04-19 

Détection des ports#...

 Dispositif trouvé avec les caractéristiques suivantes:
 Device ID: 01200816
 Nom du port: COM3
 Port baudrate: 2000000
Ouverture du port...
Dispositif: AW-A2, avec ID: 01200816 ouvert. 

Dispositif en mode configuration...

Les fréquences disponibles sont:
0: 120
1: 100
2: 80
3: 60
4: 40
Fréquence d'acquisition choisi:  120 

Les canaux disponibles sont: 
0: 11
1: 12
2: 13
3: 14
4: 15
5: 16
6: 17
7: 18
8: 19
9: 20
10: 21
11: 22
12: 23
13: 24
14: 25
Canal de radio choisi:  11 

Dispositifs rejetés:
Dispositifs acceptés:
0 - 00B43CDE
Putting devices into measurement mode...

First data packet:  [0. 0. 0.] 

Rotation matrix: [] 

Waiting 10 seconds before acquiring data...

Starting acquisition...

Other data packet:  [0.02831438 0.76626303 9.73912805] 

Rotation matrix:  [[ 0.10491401 -0.9927375   0.0588668 ]
 [ 0.99444798  0.104242   -0.0143814 ]

In [6]:
import threading as th
def key_capture_thread():
    global keep_going
    input("Press enter to stop recording.")
    keep_going = False

In [35]:
try:
    # Extraire la version du Xs Device Api employé

    print("Creation d'un objet XsControl...")
    control = xda.XsControl_construct()
    assert(control is not 0)

    xdaVersion = xda.XsVersion()
    xda.xdaVersion(xdaVersion)
    print("Utilisation de la version XDA: %s \n" % xdaVersion.toXsString())


    print("Détection des ports#...\n")
    ports_Scan = xda.XsScanner_scanPorts(0,100,True,True)

    # Détection des dispositifs MTw ou Station Awinda
    Ports = xda.XsPortInfo()
    for i in range(ports_Scan.size()):
        if ports_Scan[i].deviceId().isWirelessMaster() or ports_Scan[i].deviceId().isAwindaXStation():
            
            Ports = ports_Scan[i]
            break

    if Ports.empty():
        raise RuntimeError("Échec. Aucun dispositif a été detecté.")

    devId = Ports.deviceId()
    print(" Dispositif trouvé avec les caractéristiques suivantes:")
    print(" Device ID: %s" % devId.toXsString())
    print(" Nom du port: %s" % Ports.portName())
    print(" Port baudrate: %s" % Ports.baudrate())

    print("Ouverture du port...")
    if not control.openPort(Ports.portName(), Ports.baudrate()):
        raise RuntimeError("Échec de l'ouverture du port.")

    device = control.device(devId)

    assert(device is not 0)
    print("Dispositif: %s, avec ID: %s ouvert. \n" %(device.productCode(), device.deviceId().toXsString()))

    callback = XdaCallback()
    
    device.addCallbackHandler(callback)

    # Mettre le dispositif en mode configuration
    print("Dispositif en mode configuration...\n")
    if not device.gotoConfig():
        raise RuntimeError("Échec. Le dispositif n'a as pu être mis en mode configuration.")

    device.setOptions(xda.XSO_Orientation + xda.XSO_Calibrate + xda.XSO_RetainLiveData, 0)


    # Acquisition des fréquences pour la mise à jour des données
    fq_support = device.supportedUpdateRates(xda.XDI_None)
    index_fq = []

    while index_fq == []:
        print("Les fréquences disponibles sont:")
        print('\n'.join('{}: {}'.format(*k) for k in enumerate(fq_support)))
        choix_fq = input("Veuillez choisir une fréquence d'acquisition.")
        if choix_fq == []:
            continue
        index_fq = fq_support.index(int(choix_fq))
    print("Fréquence d'acquisition choisi: ", fq_support[index_fq], "\n")

    # Mise à jour de la fréquence d'acquisition
    if not device.setUpdateRate(fq_support[index_fq]):
        raise RuntimeError("Échec de la mise à jour du update rate.")

    
    # Acquisition du canal radio
    canaux_radio = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
    index_radio = []

    while index_radio == []:
        print("Les canaux disponibles sont: ")
        print('\n'.join('{}: {}'.format(*k) for k in enumerate(canaux_radio)))
        choix_radio = input("Veuillez choisir un canal de radio.")
        if choix_radio == []:
            continue
        index_radio = canaux_radio.index(int(choix_radio))
    print("Canal de radio choisi: ", canaux_radio[index_radio], "\n")

    # Mise à jour du canal radio
    try:
        device.enableRadio(canaux_radio[index_radio])
    except ValueError:
        print("La radio est encore activée, veuillez débrancher l'appareil de l'ordinateur et reessayer.")


    input('\n Undock the MTw devices from the Awinda station and wait until the devices are connected (synced leds), then press enter... \n')

    # Vérifier les dispositifs détectées (nombre de capteurs MTw)
    children = device.children()

    devIdAll = []
    for i in range(len(children)):
        devIdAll.append(children[i].deviceId()) 

    devicesUsed,devIdUsed,nDevs = checkConnectedSensors(devIdAll)

    ##################

    # Mis les capteurs MTw en mode acquisition des données
    print("Putting devices into measurement mode...\n")
    
    if not device.gotoMeasurement():
        raise RuntimeError("Could not put device into measurement mode. Aborting.")
    
  
    time.sleep(15)
    print("Starting acquisition...\n")
    
    keep_going = True
    s = []
    acc = []
    pack = []
    i = 0
    th.Thread(target=key_capture_thread, args=(), name='key_capture_thread', daemon=True).start()
    while keep_going:
        pack.append(children[0].getDataPacketCount())
        packet = children[0].getDataPacketByIndex(pack[-1])
        acc.append(packet.calibratedData().m_acc)
        s.append(packet.orientationMatrix())
        time.sleep(1/fq_support[index_fq])

    print("Removing callback handler...")
    device.removeCallbackHandler(callback)


    ##################

    exit = int(input('\n Appuyez sur la touche 0 pour quitter le mode acquisition... \n'))
    if exit == 0:
        stopAll()
    
except RuntimeError as error:
    print(error)
    sys.exit(1)

except:
    print("An unknown fatal error has occured. Aborting.")
    sys.exit(1)
    
else:
    print("Successful exit.")

Creation d'un objet XsControl...
Utilisation de la version XDA: 2021.0.0 build 5908 rev 109668 built on 2021-04-19 

Détection des ports#...

 Dispositif trouvé avec les caractéristiques suivantes:
 Device ID: 01200816
 Nom du port: COM3
 Port baudrate: 2000000
Ouverture du port...
Dispositif: AW-A2, avec ID: 01200816 ouvert. 

Dispositif en mode configuration...

Les fréquences disponibles sont:
0: 120
1: 100
2: 80
3: 60
4: 40
Fréquence d'acquisition choisi:  80 

Les canaux disponibles sont: 
0: 11
1: 12
2: 13
3: 14
4: 15
5: 16
6: 17
7: 18
8: 19
9: 20
10: 21
11: 22
12: 23
13: 24
14: 25
Canal de radio choisi:  24 

Dispositifs rejetés:
Dispositifs acceptés:
0 - 00B43B3E
Putting devices into measurement mode...

Starting acquisition...

Removing callback handler...
Arrêt de l'enregistrement, mise du dispositif en mode configuration.
Fermeture du fichier mtb
Fermeture du port
Successful exit.


In [5]:
stopAll()

Arrêt de l'enregistrement, mise du dispositif en mode configuration.
Fermeture du fichier mtb
Fermeture du port


In [41]:
print(s[100:120])

[array([], shape=(0, 0), dtype=float64), array([], shape=(0, 0), dtype=float64), array([[ 0.87290703, -0.48773761, -0.01205596],
       [ 0.44466861,  0.80550818, -0.39169683],
       [ 0.20075645,  0.33655401,  0.92001535]]), array([], shape=(0, 0), dtype=float64), array([[ 0.87699776, -0.48022355, -0.0161331 ],
       [ 0.43457133,  0.80705631, -0.39975977],
       [ 0.20499437,  0.34357744,  0.91647796]]), array([], shape=(0, 0), dtype=float64), array([[ 0.88625294, -0.46253626, -0.02481823],
       [ 0.41125651,  0.81038821, -0.41729968],
       [ 0.21312863,  0.35962641,  0.90842998]]), array([[ 0.88810555, -0.4588118 , -0.02757297],
       [ 0.40564021,  0.81056885, -0.42241468],
       [ 0.21615863,  0.36396412,  0.90598321]]), array([[ 0.88919196, -0.45655521, -0.02991657],
       [ 0.40166835,  0.8102593 , -0.42678144],
       [ 0.21908947,  0.36747409,  0.90385983]]), array([[ 0.89130738, -0.45210486, -0.0342396 ],
       [ 0.39425589,  0.81012098, -0.43389663],
       [ 0.22